In [43]:
import pandas as pd
import json
import glob

In [44]:
# bs_all_accs = set()
report_types_bs_is = {"CDKT":{},
                        "KQKD":{}
                     }
report_types_cf = {"LC": set()
                    }

In [45]:
for report_type, report_all_accounts in report_types_bs_is.items():
    lookup_dict = {}
    
    for file in glob.glob(f'localData/financeInfo/*{report_type}_Annual*.json'):
        with open(file, 'r') as jsonfile:
            try:
                j = json.load(jsonfile)
                for report_fullname, report_content in j[1].items():
                    for acc in report_content:
                        ### Each account with its ID, and details as [name, level, parent, name_vi]
                        n_strip = acc['NameEn'].strip()
                        n_lower = n_strip.lower()
                        acc_id = acc['ReportNormID']
                        if acc_id not in lookup_dict.keys():
                            lookup_dict[acc_id] = n_lower
            except:
                print("EXCEPTION IN LOOKUP DICT")
    
    print("lookup dict", lookup_dict)
                        
    
    for file in glob.glob(f'localData/financeInfo/*{report_type}_Annual*.json'):
        with open(file, 'r') as jsonfile:
            try:
                j = json.load(jsonfile)
                for report_fullname, report_content in j[1].items():
                    for acc in report_content:
                        ### Each account with its ID, and details as [name, parent id, parent name]
                        n_strip = acc['NameEn'].strip()
                        n_lower = n_strip.lower()
                        acc_id = acc['ReportNormID']
                        parent_id = acc['ParentReportNormID']
                        # Look up the acc's parent name and add acc details
                        parent_n = lookup_dict[parent_id]
                        acc_details = [n_lower, parent_id, parent_n]

                        if acc_id not in report_all_accounts.keys():
                            report_all_accounts[acc_id] = [acc_details]
                        else:
                            ### Check if the account is in the level list
                            if acc_details not in report_all_accounts[acc_id]:
                                report_all_accounts[acc_id].append(acc_details)
            except:
                print("exception")
                        
    report_all_accounts = dict(sorted(report_all_accounts.items()))
    with open(f'schema/{report_type}_all_accounts.json', 'w') as writefile:
        json.dump(report_all_accounts, writefile, ensure_ascii=False, indent=4)

EXCEPTION IN LOOKUP DICT
lookup dict {2995: 'assets', 3000: 'a. short-term assets', 3003: 'i. cash and cash equivalents', 3018: '1. cash', 3019: '2. cash equivalents', 3004: 'ii.  short-term financial investments', 3020: '1. available for sale securities', 3021: '2. provision for diminution in value of available for sale securities (*)', 5297: '3. held to maturity investments', 3005: 'iii. short-term receivables', 3022: '1. short-term trade accounts receivable', 3023: '2. short-term prepayments to suppliers', 3024: '3. short-term inter-company receivables', 3025: '4. construction contract progress receipts due from customers', 5300: '5. short-term loan receivables', 3026: '6. other short-term receivables', 3079: '7. provision for short-term doubtful debts (*)', 5299: '8. assets awaiting resolution', 3006: 'iv. inventories', 3027: '1. inventories', 3080: '2. provision for decline in value of inventories', 3007: 'v. other short-term assets', 3028: '1. short-term prepayments', 3029: '2. v

In [47]:
for k, v in report_types_bs_is['CDKT'].items():
    if len(v[0]) > 3:
        print ("!!")